# Neural network utils

In [ ]:
import pandas as pd # Vous devez normalement mieux connaître cette librairie

In [ ]:
# Te souviens-tu comment lire un csv ?
data = pd.read_csv("./clean_weather.csv", index_col=0)
data = data.ffill()
data.plot.scatter("tmax", "tmax_tomorrow")


In [ ]:
data.corr()

In [ ]:
import matplotlib.pyplot as plt

data.plot.scatter("tmax", "tmax_tomorrow")

prediction = lambda x, w1=.82, b=11.99: x * w1 + b

plt.plot([30, 120], [prediction(30), prediction(120)], 'green')

In [ ]:
import numpy as np

def mse(actual, predicted):
    return np.mean((actual - predicted) ** 2)

print(mse(data["tmax_tomorrow"], prediction(data["tmax"])))
print(mse(data["tmax_tomorrow"], prediction(data["tmax"], .82, 13)))

In [ ]:
tmax_bins = pd.cut(data["tmax"], 25) # Classer les données dans des bacs
tmax_bins

In [ ]:
ratios = (data["tmax_tomorrow"] - 11.99) / data["tmax"]
binned_ratio = ratios.groupby(tmax_bins, observed=True).mean()

binned_tmax = data["tmax"].groupby(tmax_bins, observed=True).mean()
plt.scatter(binned_tmax, binned_ratio)

Comme nous pouvons l'apercevoir la température n'évolue pas de façon linéaire. La régression linéaire seule ne suffira pas.   

Pour résoudre ce problème, notre réseau de neurones va passer par 3 étapes clés:   
1. Une transformation non-linéaire au-dessus de la transformation linéaire
2. Plusieurs layers, qui vont chacune récupérer les interactions entre les caractéristiques
3. Plusieurs unités cachées par couche dont chacune a des transformations linéaires et non-linéaires


## Fonction d'activation - ReLU
**Equation de la régression linéaire:**   
    
$y = wx + b$

non-linéaire    
$y = relu(wx + b)$

*w = weight (poids)*   
*b = bias (biais)*

In [ ]:
temps = np.arange(-50, 50) # Plage d'intervalle de test compris entre -50 et 50

plt.plot(temps, np.maximum(0, prediction(temps))) # Représentation graphique de notre fonction d'activation

Notre fonction d'activation n'est plutôt pas mal mais il y a encore un soucis car dans l'exemple du graphique précédent, pour une température de -40° nous allons prédire 0° pour le lendemain. Ce qui sera généralement une très mauvaise prédiction !

C'est donc pour cela que notre réseau de neurones aura besoin de plusieurs couches.

$\hat{y} = w_{2} relu(w_{1}x + b_{1}) + b_2$

$relu(w_{1}x + b_{1})$   étant notre première couche

$w_{2} * output + b_2$   est notre deuxième couche

In [ ]:
temps = np.arange(-50, 50) # On re initie une intervalle de test

layer1 = np.maximum(0, prediction(temps)) # Première couche
layer2 = prediction(layer1, .5, 10) # Deuxième couche ( poids = 0.5 et biais = 10)

plt.plot(temps, layer2)

plt.ylim((0,40)) # Limiter la taille du graphique sur l'axe y

# Par la suite le réseau de neurone apprendra lui même ses poids et biais à assigner

Nous pouvons apercevoir une légère amélioration car lorsque la température est négative, celle prédite le lendemain sera maintenant de 10. Le biais s'ajustera seul pour trouver une valeur plus cohérente. Mais nous appliquons toujours une constante pour toute température inférieure à 0°. Pour résoudre ce problème nous allons ajouter plusieurs couches d'unités.

## Multiple hidden units

### 1. Aperçu des layers

In [ ]:
layer1_1 = np.maximum(0, prediction(temps))

layer1_2 = np.maximum(0, prediction(temps, .1, 10)) # Exemple

layer1_3 = np.maximum(0, prediction(temps, 2, -50)) # Exemple

layer2 = layer1_1 * .1 + layer1_2 * .3 + layer1_3 * .4 + 20

# plt.plot(temps, layer1_1)
# plt.plot(temps, layer1_2)
# plt.plot(temps, layer1_3)
# plt.plot(temps, layer1_1 + layer1_2 + layer1_3)
plt.plot(temps, layer2)

Nous pouvons observer qu'en fonction de nos couches et de l'ajustement de nos biais notre réseau de neurones peut comprendre des interactions plus complexes.

### 2. Création des matrices

En mathématiques, les matrices sont des tableaux d'éléments qui servent à interpréter en termes calculatoires, les résultats théoriques de l'algèbre linéaire et même de l'algèbre bilinéaire.

<img src="assets/matrices2.png" alt="Exemple de matrice" style="width:779px;height:401px;">

La multiplication entre matrices se passe comme ça:

<img src="assets/matrice_mul.gif" alt="Matrice mul" style="width:600px;height:300px;">


In [ ]:
from tsensor import explain as exp # Cette librairie permet d'avoir plus d'infos sur les matrices calculées

my_input = np.array([[80], [90], [100], [-20], [-10]])

l1_weights = np.array([[.82, .1]])

l1_bias = np.array([[11.99, 10]])

with exp():
    l1_output = my_input @ l1_weights + l1_bias

## Forward pass

In [ ]:
print(l1_output, end="\n\n")
l1_activated = np.maximum(l1_output, 0) # Fonction d'activation (ReLU)
print(l1_activated)

Notre fonction d'activation a bien affecté notre matrice. La valeur négative (-4) a été mise à 0 comme le prévoit ReLU.

$layer_{1}=relu(XW_{1} + B_{1})$

$\hat{y}=W_{2}relu(XW_{1} + B_{1}) + B_{2}$

In [ ]:
l2_weights = np.array([
    [.5],
    [.2]
])

l2_bias = np.array([[5]])

with exp():
    output = l1_activated @ l2_weights + l2_bias
output

In [ ]:
tmax = np.array([[80], [90], [100], [-20], [-10]])
tmax_tomorrow = np.array([[83], [89], [95], [-22], [-9]])
tmax_tomorrow

In [ ]:
def mse(actual, predicted):
    return(actual - predicted) ** 2

In [ ]:
mse(tmax_tomorrow, output)

In [ ]:
def mse_grad(actual, predicted):
    return predicted - actual

In [ ]:
mse_grad(tmax_tomorrow, output)

## Back-propagation
Mettre à jour nos poids et biais

In [ ]:
output_gradient = mse_grad(tmax_tomorrow, output)

In [ ]:
with exp():
    l2_w_gradient = l1_activated.T @ output_gradient

l2_w_gradient

Pour trouver les bons poids à mettre nous devont calculé la dérivé partielle de la perte par rapport à la deuxième matrice de poids.

$\frac{\partial L}{\partial XW_{2}}$
La formule ci-dessus indique comment une petite variation dans la sortie de la couche cachée affecte la fonction de perte. Cette information est utilisée pour propager l'erreur de la couche de sortie vers la couche cachée lors de la rétropropagation.


Comment la calculer:   
 
 $$\frac{\partial L}{\partial W_{2}}=\partial L\frac{\partial (XW_{2})}{\partial W_{2}}$$

In [ ]:
# Calcul de la dérivée par rapport au biais
with exp():
    l2_b_gradient = np.mean(output_gradient, axis=0)
l2_b_gradient

Nous allons maintenant utiliser l'[algorithme du gradient (Gradient descent)](https://fr.wikipedia.org/wiki/Algorithme_du_gradient) pour mettre à jour nos poids et biais de la seconde couche.

Nous allons avoir besoin d'un **learning rate (taux d'apprentissage)** qui va permettre à notre *gradient descent* de converger vers la sortie souhaité. Plus il sera haut, plus il sera difficile pour lui de converger. Plus il sera bas plus il mettra du temps à converger. Il faut donc trouver une valeur souhaitable, on ne peut la connaître que en testant différentes valeures. C'est ce qu'on appelle un **hyperparamètre**.

In [ ]:
learning_rate = 1e-5

with exp():
    l2_bias = l2_bias - l2_b_gradient * learning_rate
    l2_weights = l2_weights - l2_w_gradient * learning_rate

l2_weights

Après avoir calculé les poids et biais de la seconde couche nous pouvons maintenant calculer ceux de la première couche !

In [ ]:
with exp():
    l1_activated_gradient = output_gradient @ l2_weights.T
l1_activated_gradient

In [ ]:
temps = np.arange(-50, 50)
plt.plot(temps, np.maximum(0, temps))

In [ ]:
activation = np.maximum(0, temps)
plt.plot(temps[1:], activation[1:] - np.roll(activation, 1)[1:])

In [ ]:
with exp():
    l1_output_gradient = l1_activated_gradient * np.heaviside(l1_output, 0)
l1_output_gradient

In [ ]:
# Back-propagation
l1_w_gradient = my_input.T @ l1_output_gradient
l1_b_gradient = np.mean(l1_output_gradient, axis=0)

# Gradient descent
l1_weights -= l1_w_gradient * learning_rate
l1_bias -= l1_b_gradient * learning_rate

In [ ]:
l1_weights

In [ ]:
l1_bias

### Résumé de ce que nous avons pu voir ici...

1. Lancer le forward pass, et récupérer l'output:
2. Calculer le gradient par rapport aux sorties du réseau. (fonction `mse_grad`)
3. Pour chaque couche (layer) du réseau:
    - Calculer le gradient par rapport à la non-linéarité de la sortie (si la couche a de la non-linéarité)
    - Calculer le gradient par rapport aux poids
    - Calculer le gradient par rapport aux biais
    - Calculer le gradient par rapport aux entrées (inputs) de la couche
4. Mettre à jour les paramètres du réseau en utilisant le *Gradient descent*